In [1]:
!pip install transformers
!pip install einops
!pip install datasets


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip install rouge_score
!pip install evaluate


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import torch
from transformers import AutoModelWithLMHead, AutoTokenizer,AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
#model = AutoModelWithLMHead.from_pretrained("mrm8488/t5-base-finetuned-wikiSQL")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")

In [4]:
import pandas as pd

df=pd.read_csv('sentiment-emotion-labelled_Dell_tweets.csv')
df

,Unnamed: 0,Datetime,Tweet Id,Text,Username,sentiment,sentiment_score,emotion,emotion_score
0,0,2022-09-30 23:29:15+00:00,1575991191170342912,@Logitech @apple @Google @Microsoft @Dell @Len...,ManjuSreedaran,neutral,0.853283,anticipation,0.587121
1,1,2022-09-30 21:46:35+00:00,1575965354425131008,@MK_habit_addict @official_stier @MortalKombat...,MiKeMcDnet,neutral,0.519470,joy,0.886913
2,2,2022-09-30 21:18:02+00:00,1575958171423752203,"As @CRN celebrates its 40th anniversary, Bob F...",jfollett,positive,0.763791,joy,0.960347
3,3,2022-09-30 20:05:24+00:00,1575939891485032450,@dell your customer service is horrible especi...,daveccarr,negative,0.954023,anger,0.983203
4,4,2022-09-30 20:03:17+00:00,1575939359160750080,@zacokalo @Dell @DellCares @Dell give the man ...,heycamella,neutral,0.529170,anger,0.776124
...,...,...,...,...,...,...,...,...,...
24965,24965,2022-01-01 02:02:04+00:00,1477097760931336198,@ElDarkAngel2 @GamersNexus @Dell I wouldn't ev...,Eodart,negative,0.682981,anger,0.906309
24966,24966,2022-01-01 01:57:34+00:00,1477096631300415496,@kite_real @GamersNexus @Dell I didn't really ...,Eodart,positive,0.743940,joy,0.951701
24967,24967,2022-01-01 01:36:36+00:00,1477091355629432833,Hey @JoshTheFixer here it is....27 4K UHD USB-...,Corleone250,neutral,0.654463,anticipation,0.471185
24968,24968,2022-01-01 01:31:30+00:00,1477090070830141442,@bravadogaming @thewolfpena @Alienware @intel ...,MrTwistyyy,neutral,0.794049,anticipation,0.747014


In [5]:
df=df[['Text','emotion']]
df

,Text,emotion
0,@Logitech @apple @Google @Microsoft @Dell @Len...,anticipation
1,@MK_habit_addict @official_stier @MortalKombat...,joy
2,"As @CRN celebrates its 40th anniversary, Bob F...",joy
3,@dell your customer service is horrible especi...,anger
4,@zacokalo @Dell @DellCares @Dell give the man ...,anger
...,...,...
24965,@ElDarkAngel2 @GamersNexus @Dell I wouldn't ev...,anger
24966,@kite_real @GamersNexus @Dell I didn't really ...,joy
24967,Hey @JoshTheFixer here it is....27 4K UHD USB-...,anticipation
24968,@bravadogaming @thewolfpena @Alienware @intel ...,anticipation


In [7]:
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df, test_size=0.1, random_state=42)
df_test.shape

(2497, 2)

In [ ]:
df_test

In [8]:
from datasets import Dataset
train_dataset = Dataset.from_pandas(df_train)
test_dataset = Dataset.from_pandas(df_test)

In [9]:
train_dataset

Dataset({
    features: ['Text', 'emotion', '__index_level_0__'],
    num_rows: 22473
})

In [10]:
from datasets import concatenate_datasets


tokenized_inputs = concatenate_datasets([train_dataset, test_dataset]).map(lambda x: tokenizer(x["Text"], truncation=True), batched=True, remove_columns=['emotion', '__index_level_0__'])
max_source_length = max([len(x) for x in tokenized_inputs["input_ids"]])
print(f"Max source length: {max_source_length}")

Map:   0%|          | 0/24970 [00:00<?, ? examples/s]

Max source length: 420


In [11]:
tokenized_targets = concatenate_datasets([train_dataset, test_dataset]).map(lambda x: tokenizer(x["emotion"], truncation=True), batched=True, remove_columns=["Text",  '__index_level_0__'])
max_target_length = max([len(x) for x in tokenized_targets["input_ids"]])
print(f"Max target length: {max_target_length}")

Map:   0%|          | 0/24970 [00:00<?, ? examples/s]

Max target length: 2


In [12]:
def preprocess_function(sample,padding="max_length"):

    inputs = ["Find emotion : " + item for item in sample["Text"]]

  
    model_inputs = tokenizer(inputs, max_length=max_source_length, padding=padding, truncation=True)


    labels = tokenizer(text_target=sample["emotion"], max_length=max_target_length, padding=padding, truncation=True)

    if padding == "max_length":
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_train = train_dataset.map(preprocess_function, batched=True, remove_columns=['Text','emotion', '__index_level_0__'])
tokenized_test = test_dataset.map(preprocess_function, batched=True, remove_columns=['Text','emotion', '__index_level_0__'])
print(f"Keys of tokenized dataset: {list(tokenized_train.features)}")

Map:   0%|          | 0/22473 [00:00<?, ? examples/s]

Map:   0%|          | 0/2497 [00:00<?, ? examples/s]

Keys of tokenized dataset: ['input_ids', 'attention_mask', 'labels']


In [13]:
from transformers import DataCollatorForSeq2Seq


label_pad_token_id = -100

data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)

In [14]:
import evaluate
import nltk
import numpy as np
from nltk.tokenize import sent_tokenize
nltk.download("punkt")


metric = evaluate.load("rouge")


def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]


    preds = ["\n".join(sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(sent_tokenize(label)) for label in labels]

    return preds, labels

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\bbsur\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [15]:
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)


    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    result = {k: round(v * 100, 4) for k, v in result.items()}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    return result

In [16]:
from huggingface_hub import HfFolder
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments


training_args = Seq2SeqTrainingArguments(
    output_dir=f"/model",
    predict_with_generate=True,
    fp16=False, 
    learning_rate=5e-5,
    num_train_epochs=1
)

In [17]:
from transformers import Seq2SeqTrainingArguments, TrainingArguments,Trainer

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    compute_metrics=compute_metrics,
)

In [18]:
trainer.train()

C:\Users\bbsur\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,0.684300
1000,0.467100
1500,0.435400
2000,0.412100
2500,0.388900


TrainOutput(global_step=2810, training_loss=0.4680954929772645, metrics={'train_runtime': 63035.4408, 'train_samples_per_second': 0.357, 'train_steps_per_second': 0.045, 'total_flos': 1.2743644609732608e+16, 'train_loss': 0.4680954929772645, 'epoch': 1.0})

In [19]:
trainer.evaluate()

{'eval_loss': 0.35223594307899475,
 'eval_rouge1': 74.8498,
 'eval_rouge2': 0.0,
 'eval_rougeL': 74.9099,
 'eval_rougeLsum': 74.8498,
 'eval_gen_len': 2.0,
 'eval_runtime': 4714.8606,
 'eval_samples_per_second': 0.53,
 'eval_steps_per_second': 0.066,
 'epoch': 1.0}

In [20]:
input_question = "Find emotion : @Dell @YouTube A one month old laptop is performing so weird that even their customer service people are unable to solve the problem"
tokenized_question = tokenizer(input_question, truncation=True, padding=True, return_tensors='pt')
#input_question_ids = tokenized_question['input_ids'].to(device)
#input_attention_mask = tokenized_question['attention_mask'].to(device)
output = model.generate(input_ids=tokenized_question['input_ids'], attention_mask=tokenized_question['attention_mask'], max_length=512)

In [21]:
ans = tokenizer.decode(output[0], skip_special_tokens=True)
print(f"{ans}")

anger


In [24]:
input_question = " @Dell Why can't resolved my issue. My system was shutdown but not coming Dell customer service centre.. nothing Any help."
tokenized_question = tokenizer(input_question, truncation=True, padding=True, return_tensors='pt')
output = model.generate(input_ids=tokenized_question['input_ids'], attention_mask=tokenized_question['attention_mask'], max_length=512)

In [25]:
input_question = " @Dell Why can't resolved my issue. My system was shutdown but not coming Dell customer service centre."
tokenized_question = tokenizer(input_question, truncation=True, padding=True, return_tensors='pt')
output = model.generate(input_ids=tokenized_question['input_ids'], attention_mask=tokenized_question['attention_mask'], max_length=512)
ans = tokenizer.decode(output[0], skip_special_tokens=True)
print(f"{ans}")

anger


In [28]:
input_question ="@Dell Something about coffee and inspiration by coffee."
tokenized_question = tokenizer(input_question, truncation=True, padding=True, return_tensors='pt')
output = model.generate(input_ids=tokenized_question['input_ids'], attention_mask=tokenized_question['attention_mask'], max_length=512)

In [29]:
ans = tokenizer.decode(output[0], skip_special_tokens=True)
print(f"{ans}")

joy
